# 拉格朗日有限元

下面给出任意次拉格朗日有限元，求解带混合边界条件的 Poisson 方程的算例。给定区域 $\Omega\subset \mathbb{R}^m$，其边界 $\partial\Omega=\Gamma_D\cup\Gamma_N\cup\Gamma_R$。经典的 Poisson 方程形式如下
$$
\begin{aligned}-\Delta u &= f,\quad \text{in}\, \Omega\\
u&=g_D,\quad \text{on}\,\Gamma_D \\
\frac{\partial u}{\partial \boldsymbol{n}}&=g_N,\quad \text{on}\,\Gamma_N \\
\frac{\partial u}{\partial \boldsymbol{n}} + \kappa u&=g_R,\quad \text{on}\,\Gamma_R \end{aligned}
$$
其中 $\Gamma_D$ 代表 Dirichlet 边界，$\Gamma_N$ 代表 Neumann 边界，$\Gamma_R$ 代表 Robin 边界。 在方程两端分别乘以测试函数 $v\in H_{D,0}^1(\Omega)$（在 $\Gamma_D$ 上取值为 0 且函数及其导数都 $L^2$ 可积的函数空间），化原方程为积分形式：
$$
(\nabla u,\,\nabla v)+<\kappa 
u,\,v>_{\Gamma_R} = (f,\,v)+<g_R,\,v>_{\Gamma_R}+<g_N,\,v>_{\Gamma_N}
$$

下面以二维的 Poisson 方程为例来介绍编程过程。假定方程的真解为定义在单位正方形区域 $\Omega = [0, 1]^2$ 二元函数
$$u =\cos(\pi x)\cos(\pi y).$$
进一步假定 Robin 边界条件中的参数为 $\kappa=1$，则模型中的源项 $f$ 和边界源项 $g_N$ 和 $g_R$ 可相应推出。区域 $\Omega$ 有上下左右四边边界，我们假定上下边界为 Dirichlet 边界， 右边界为 Neumann 边界，左边界为 Robin 边界。

**第一步**，首先给出该 PDE 模型的代码实现，并建立相应的 PDE 对象。

In [2]:
from fealpy.pde.poisson_2d import CosCosData

pde = CosCosData()
domain = pde.domain()

**第二步**，生成网格，这里以 TriangleMesh 为例。

In [4]:
from fealpy.mesh import TriangleMesh
mesh = TriangleMesh.from_box(domain, nx=100, ny=100)

**第三步**，导入拉格朗日有有限元空间，并建立 $p$ 次拉格朗日有限元空间对象。

In [5]:
from fealpy.functionspace import LagrangeFESpace
p = 1 # 空间次数
space = LagrangeFESpace(mesh, p=p)

**第四步**，导入双线性型 BilnearForm  以及 Laplace 积分子和 Robin 边界积分子，并组装离散矩阵。

In [6]:
from fealpy.fem import BilinearForm
from fealpy.fem import ScalarLaplaceIntegrator      # (\nabla u, \nabla v) 
from fealpy.fem import ScalarRobinBoundaryIntegrator  # <kappa*u, v>

bform = BilinearForm(space)
# (\nabla u, \nabla v)
bform.add_domain_integrator(ScalarLaplaceIntegrator(q=p+2)) 
# <kappa u, v>
rbi = ScalarRobinBoundaryIntegrator(pde.kappa,
        threshold=pde.is_robin_boundary, q=p+2)
bform.add_boundary_integrator(rbi) 
A = bform.assembly()

**第五步**， 导入线性型 LinearForm 以及区域源项、Neumann 和 Robin 边界源项的积分子，并组装右端矩阵向量。

In [7]:
from fealpy.fem import LinearForm
from fealpy.fem import ScalarSourceIntegrator         # (f, v)
from fealpy.fem import ScalarNeumannSourceIntegrator  # <g_N, v>
from fealpy.fem import ScalarRobinSourceIntegrator    # <g_R, v>

lform = LinearForm(space)
# (f, v)
si = ScalarSourceIntegrator(pde.source, q=p+2)
lform.add_domain_integrator(si)
# <g_R, v> 
rsi = ScalarRobinSourceIntegrator(pde.robin, 
           threshold=pde.is_robin_boundary, q=p+2)
lform.add_boundary_integrator(rsi)
# <g_N, v>
nsi = ScalarNeumannSourceIntegrator(pde.neumann, 
            threshold=pde.is_neumann_boundary, q=p+2)
lform.add_boundary_integrator(nsi)
F = lform.assembly()

**第六步**，导入并处理 Dirichlet 边界条件，并求解计算误差。

In [8]:
from fealpy.fem import DirichletBC

# Dirichlet 边界条件
bc = DirichletBC(space, 
      pde.dirichlet, threshold=pde.is_dirichlet_boundary) 
uh = space.function() 
A, F = bc.apply(A, F, uh)

**第七步**，导入多重网格解法器，求解并计算误差。

In [10]:
from fealpy.solver import GAMGSolver # 几何、代数多重网格解法器

solver = GAMGSolver(ptype='W', sstep=3)
solver.setup(A) # 纯代数多重网格
# solver.setup(A, space=space, cdegree=[1, 3]) # 高次到低次空间的限制+代数多重网格
uh[:] = solver.solve(F)

L2Error = mesh.error(pde.solution, uh, q=p+2)
H1Error = mesh.error(pde.gradient, uh.grad_value, q=p+2)
print('L2Error:', L2Error)
print('H1Error:', H1Error)

run setup with time: 0.04326722199948563
iter   1
iter   2
iter   3
iter   4
iter   5
iter   6
iter   7
iter   8
iter   9
iter  10
0
run solve with time: 1.5513344810005947
L2Error: 0.00011937504415731966
H1Error: 0.034891662108901975
